# Project 1 

## Group One: Covid 19 | Part One: Data Cleansing 

Team Members: Michael Bien, Lupita Lopez, Jinah Porter, Debra Wu

### Research Questions to Answer:

1. WORLD: Compare COVID deaths by GDP by Country: Does a higher GDP result in a lower number of COVID deaths?

2. US: Examine death rates by age group and determine is children under 10 die at a lower rate, adults 60+ at a higher rate

3. US: Compare cases/deaths by state

4. US: Does pre-COVID state GDP related to the cases/deaths in a given state

5. March shutdown, 4th of July and Memorial Day: compare 2 months of shutdown vs. reopening: how did reopening affect case and death rates? 

6. US: Is there a relationship between COVID cases/deaths and gender?

7. US: Is there a relationship between COVID cases/deaths and ethnicity?


In [1]:
#import
import pandas as pd
import os
import requests
import numpy as np
import requests
from pprint import pprint

In [2]:
url = 'https://api.covidtracking.com/v1/states/current.json'

print(url)


https://api.covidtracking.com/v1/states/current.json


In [3]:
print(requests.get(url))

<Response [200]>


In [4]:
response = requests.get(url).json()

pprint(response)



[{'checkTimeEt': '09/14 23:59',
  'commercialScore': 0,
  'dataQualityGrade': 'A',
  'date': 20200915,
  'dateChecked': '2020-09-15T03:59:00Z',
  'dateModified': '2020-09-15T03:59:00Z',
  'death': 44,
  'deathConfirmed': 44,
  'deathIncrease': 0,
  'deathProbable': None,
  'fips': '02',
  'grade': '',
  'hash': 'b7d286bbeacaeef7cbaed188fba49cfe43c44450',
  'hospitalized': None,
  'hospitalizedCumulative': None,
  'hospitalizedCurrently': 44,
  'hospitalizedIncrease': 0,
  'inIcuCumulative': None,
  'inIcuCurrently': None,
  'lastUpdateEt': '9/15/2020 03:59',
  'negative': 399811,
  'negativeIncrease': 3668,
  'negativeRegularScore': 0,
  'negativeScore': 0,
  'negativeTestsAntibody': None,
  'negativeTestsPeopleAntibody': None,
  'negativeTestsViral': 400207,
  'onVentilatorCumulative': None,
  'onVentilatorCurrently': 8,
  'pending': None,
  'posNeg': 407124,
  'positive': 7313,
  'positiveCasesViral': 7313,
  'positiveIncrease': 43,
  'positiveScore': 0,
  'positiveTestsAntibody': No

  'commercialScore': 0,
  'dataQualityGrade': 'A+',
  'date': 20200915,
  'dateChecked': '2020-09-14T10:00:00Z',
  'dateModified': '2020-09-14T10:00:00Z',
  'death': 534,
  'deathConfirmed': None,
  'deathIncrease': 0,
  'deathProbable': None,
  'fips': '20',
  'grade': '',
  'hash': 'f2f2ff46eba821dc5c314d7e7dbf7ae2764b34b7',
  'hospitalized': 2572,
  'hospitalizedCumulative': 2572,
  'hospitalizedCurrently': 192,
  'hospitalizedIncrease': 0,
  'inIcuCumulative': 703,
  'inIcuCurrently': 50,
  'lastUpdateEt': '9/14/2020 10:00',
  'negative': 408482,
  'negativeIncrease': 0,
  'negativeRegularScore': 0,
  'negativeScore': 0,
  'negativeTestsAntibody': None,
  'negativeTestsPeopleAntibody': None,
  'negativeTestsViral': None,
  'onVentilatorCumulative': 221,
  'onVentilatorCurrently': 15,
  'pending': None,
  'posNeg': 458381,
  'positive': 49899,
  'positiveCasesViral': None,
  'positiveIncrease': 0,
  'positiveScore': 0,
  'positiveTestsAntibody': None,
  'positiveTestsAntigen': None,

 {'checkTimeEt': '09/15 05:00',
  'commercialScore': 0,
  'dataQualityGrade': 'B',
  'date': 20200915,
  'dateChecked': '2020-09-15T09:00:00Z',
  'dateModified': '2020-09-15T09:00:00Z',
  'death': 438,
  'deathConfirmed': None,
  'deathIncrease': 2,
  'deathProbable': None,
  'fips': '33',
  'grade': '',
  'hash': '2bf13543c208ffefcf8c0972dcd802118a8989da',
  'hospitalized': 722,
  'hospitalizedCumulative': 722,
  'hospitalizedCurrently': 9,
  'hospitalizedIncrease': 1,
  'inIcuCumulative': 230,
  'inIcuCurrently': None,
  'lastUpdateEt': '9/15/2020 09:00',
  'negative': 224710,
  'negativeIncrease': 608,
  'negativeRegularScore': 0,
  'negativeScore': 0,
  'negativeTestsAntibody': None,
  'negativeTestsPeopleAntibody': None,
  'negativeTestsViral': None,
  'onVentilatorCumulative': None,
  'onVentilatorCurrently': None,
  'pending': None,
  'posNeg': 232458,
  'positive': 7748,
  'positiveCasesViral': 7748,
  'positiveIncrease': 34,
  'positiveScore': 0,
  'positiveTestsAntibody': Non

  'death': 19,
  'deathConfirmed': 19,
  'deathIncrease': 0,
  'deathProbable': None,
  'fips': '78',
  'grade': '',
  'hash': '3379b5f26b9605fbd4e055aea96586ebdcbfb8d3',
  'hospitalized': None,
  'hospitalizedCumulative': None,
  'hospitalizedCurrently': None,
  'hospitalizedIncrease': 0,
  'inIcuCumulative': None,
  'inIcuCurrently': None,
  'lastUpdateEt': '9/15/2020 06:34',
  'negative': 17320,
  'negativeIncrease': 190,
  'negativeRegularScore': 0,
  'negativeScore': 0,
  'negativeTestsAntibody': None,
  'negativeTestsPeopleAntibody': None,
  'negativeTestsViral': None,
  'onVentilatorCumulative': None,
  'onVentilatorCurrently': None,
  'pending': 17,
  'posNeg': 18545,
  'positive': 1225,
  'positiveCasesViral': None,
  'positiveIncrease': 4,
  'positiveScore': 0,
  'positiveTestsAntibody': None,
  'positiveTestsAntigen': None,
  'positiveTestsPeopleAntibody': None,
  'positiveTestsPeopleAntigen': None,
  'positiveTestsViral': None,
  'recovered': 1159,
  'score': 0,
  'state': 

In [5]:
len(response)
state = []
num_state = range(0,56)
date = []
death = []


for i in num_state:
    (print(response[i]['state']))
    state.append(response[i]['state'])
    (print(response[i]['date']))
    date.append(response[i]['date'])
    (print(response[i]['deathConfirmed']))
    death.append(response[i]['death'])


AK
20200915
44
AL
20200915
2253
AR
20200915
1003
AS
20200915
None
AZ
20200915
5071
CA
20200915
None
CO
20200915
None
CT
20200915
3590
DC
20200915
None
DE
20200915
546
FL
20200915
12946
GA
20200915
None
GU
20200915
None
HI
20200915
None
IA
20200915
None
ID
20200915
382
IL
20200915
8332
IN
20200915
3235
KS
20200915
None
KY
20200915
1065
LA
20200915
5108
MA
20200915
9016
MD
20200915
3706
ME
20200915
136
MI
20200915
6612
MN
20200915
1927
MO
20200915
None
MP
20200915
2
MS
20200915
2543
MT
20200915
None
NC
20200915
3111
ND
20200915
132
NE
20200915
None
NH
20200915
None
NJ
20200915
14254
NM
20200915
None
NV
20200915
None
NY
20200915
None
OH
20200915
4207
OK
20200915
None
OR
20200915
None
PA
20200915
None
PR
20200915
385
RI
20200915
None
SC
20200915
2943
SD
20200915
None
TN
20200915
2050
TX
20200915
None
UT
20200915
None
VA
20200915
2691
VI
20200915
19
VT
20200915
58
WA
20200915
2015
WI
20200915
1220
WV
20200915
278
WY
20200915
None


In [6]:
clean_df = pd.DataFrame({'State': state,
                         'Date': date,
                        'Number of Deaths': death})

clean_df

states_df = clean_df.drop(clean_df.index[[3, 12, 27, 42, 50]])

new = states_df.reset_index().drop(columns='index')

ok = new.replace({'AK': 'Alaska', 'AL':'Alabama', 'AR': 'Arkansas', 'AZ':'Arizona',
                 'CA': 'California', 'CO': 'Colorado','CT': 'Conneticut',
                 'DC': 'District of Columbia', 'DE': 'Delaware',
                 'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii',
                 'IA': 'Iowa', 'ID': 'Idaho', 'IL': 'Illinois',
                 'IN': 'Indiana', 'KS': 'Kansas', 'KY': 'Kentucky',
                 'LA': 'Louisiana', 'MA': 'Massachusetts', 'MD': 'Maryland',
                 'ME': 'Maine', 'MI': 'Michigan', 'MN': 'Minnesota',
                 'MO': 'Missouri', 'MS': 'Mississippi', 'MT': 'Montana',
                 'NC': 'North Carolina', 'ND': 'North Dakota', 'NE': 'Nebraska',
                 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico',
                 'NV': 'Nevada', 'NY': 'New York', 'OH': 'Ohio',
                 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania',
                 'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota',
                 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah',
                 'VA': 'Virginia', 'VT': 'Vermont', 'WA': 'Washington',
                 'WV': 'West Virigina', 'WI': 'Wisconsin', 'WY': 'Wyoming'})

ok

,State,Date,Number of Deaths
0,Alaska,20200915,44
1,Alabama,20200915,2387
2,Arkansas,20200915,1010
3,Arizona,20200915,5344
4,California,20200915,14451
5,Colorado,20200915,1894
6,Conneticut,20200915,4485
7,District of Columbia,20200915,616
8,Delaware,20200915,618
9,Florida,20200915,12946


In [7]:
#get the data from the CSV file for 2018 GDP Data

f_path = os.path.join('Resources', 'download (1).csv')

# col_list = ['SQGDP2 Gross domestic product (GDP) by state 1/']
gdp_df = pd.read_csv(f_path)
# quotechar='"'
# , sep=';'
gdp_df.head(25)




,GeoFips,GeoName,LineCode,Description,2018:Q1,2018:Q2,2018:Q3,2018:Q4
0,0,United States *,1.0,All industry total,20163159,20510177,20749752,20897804
1,0,United States *,2.0,Private industries,17654891,17977475,18189960,18320015
2,0,United States *,3.0,"Agriculture, forestry, fishing and hunting",169446,172100,160510,163799
3,0,United States *,6.0,"Mining, quarrying, and oil and gas extra...",328770,346432,361288,350091
4,0,United States *,10.0,Utilities,322592,326273,323103,331440
5,0,United States *,11.0,Construction,824254,835302,845371,851643
6,0,United States *,12.0,Manufacturing,2274772,2308125,2337156,2364714
7,0,United States *,13.0,Durable goods manufacturing,1272352,1286168,1305727,1321374
8,0,United States *,25.0,Nondurable goods manufacturing,1002420,1021956,1031428,1043341
9,0,United States *,34.0,Wholesale trade,1184398,1200091,1217788,1246640


In [8]:
gdp_df.isnull().any()

gdp_df.count()

gdp_df[gdp_df.isnull().any(axis=1)]

clean_df = gdp_df.dropna(how='any')

clean_df.count()

clean_df.head()

,GeoFips,GeoName,LineCode,Description,2018:Q1,2018:Q2,2018:Q3,2018:Q4
0,0,United States *,1.0,All industry total,20163159,20510177,20749752,20897804
1,0,United States *,2.0,Private industries,17654891,17977475,18189960,18320015
2,0,United States *,3.0,"Agriculture, forestry, fishing and hunting",169446,172100,160510,163799
3,0,United States *,6.0,"Mining, quarrying, and oil and gas extra...",328770,346432,361288,350091
4,0,United States *,10.0,Utilities,322592,326273,323103,331440


In [9]:
#drop geofips & line code, not sure we need them
new_df = clean_df[['GeoName', 'Description', '2018:Q1', '2018:Q2', '2018:Q3', '2018:Q4']]

In [10]:
new_df

,GeoName,Description,2018:Q1,2018:Q2,2018:Q3,2018:Q4
0,United States *,All industry total,20163159,20510177,20749752,20897804
1,United States *,Private industries,17654891,17977475,18189960,18320015
2,United States *,"Agriculture, forestry, fishing and hunting",169446,172100,160510,163799
3,United States *,"Mining, quarrying, and oil and gas extra...",328770,346432,361288,350091
4,United States *,Utilities,322592,326273,323103,331440
...,...,...,...,...,...,...
1438,Far West,Health care and social assistance,258781.8,262129,265730.2,268916.8
1439,Far West,"Arts, entertainment, and recreation",52251.6,55032.1,55224.1,56127.1
1440,Far West,Accommodation and food services,143296.6,145704.5,147121.8,150429
1441,Far West,Other services (except government and go...,77503.7,78638.8,78846.7,80519.2


In [20]:
# quarter_df = new_df[['2018:Q1', '2018:Q2', '2018:Q3', '2018:Q4']]

values_df = new_df[new_df['2018:Q1'] != '(D)'][['2018:Q1', '2018:Q2', '2018:Q3', '2018:Q4']].astype(float)

merge_df = new_df.merge(values_df, left_index=True, right_index=True)

merge_df.dtypes

#remove columns 
first_org = merge_df.drop(columns = ['2018:Q1_x', '2018:Q2_x', '2018:Q3_x', '2018:Q4_x'])

#rename columns 
new_df = first_org.rename(columns={"2018:Q1_y": "2018:Q1", "2018:Q2_y": "2018:Q2","2018:Q3_y": "2018:Q3", "2018:Q4_y": "2018:Q4"})

new_df

/Users/debrawu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,GeoName,Description,2018:Q1,2018:Q2,2018:Q3,2018:Q4
0,United States *,All industry total,20163159.0,20510177.0,20749752.0,20897804.0
1,United States *,Private industries,17654891.0,17977475.0,18189960.0,18320015.0
2,United States *,"Agriculture, forestry, fishing and hunting",169446.0,172100.0,160510.0,163799.0
3,United States *,"Mining, quarrying, and oil and gas extra...",328770.0,346432.0,361288.0,350091.0
4,United States *,Utilities,322592.0,326273.0,323103.0,331440.0
...,...,...,...,...,...,...
1438,Far West,Health care and social assistance,258781.8,262129.0,265730.2,268916.8
1439,Far West,"Arts, entertainment, and recreation",52251.6,55032.1,55224.1,56127.1
1440,Far West,Accommodation and food services,143296.6,145704.5,147121.8,150429.0
1441,Far West,Other services (except government and go...,77503.7,78638.8,78846.7,80519.2


In [23]:
new_df['Total GDP'] = new_df.sum(axis = 1)

new_df

,GeoName,Description,2018:Q1,2018:Q2,2018:Q3,2018:Q4,Total GDP
0,United States *,All industry total,20163159.0,20510177.0,20749752.0,20897804.0,164641784.0
1,United States *,Private industries,17654891.0,17977475.0,18189960.0,18320015.0,144284682.0
2,United States *,"Agriculture, forestry, fishing and hunting",169446.0,172100.0,160510.0,163799.0,1331710.0
3,United States *,"Mining, quarrying, and oil and gas extra...",328770.0,346432.0,361288.0,350091.0,2773162.0
4,United States *,Utilities,322592.0,326273.0,323103.0,331440.0,2606816.0
...,...,...,...,...,...,...,...
1438,Far West,Health care and social assistance,258781.8,262129.0,265730.2,268916.8,2111115.6
1439,Far West,"Arts, entertainment, and recreation",52251.6,55032.1,55224.1,56127.1,437269.8
1440,Far West,Accommodation and food services,143296.6,145704.5,147121.8,150429.0,1173103.8
1441,Far West,Other services (except government and go...,77503.7,78638.8,78846.7,80519.2,631016.8


In [24]:
new_df.dtypes



GeoName         object
Description     object
2018:Q1        float64
2018:Q2        float64
2018:Q3        float64
2018:Q4        float64
Total GDP      float64
dtype: object

In [25]:
new_df.head()

## change to int and then add 

,GeoName,Description,2018:Q1,2018:Q2,2018:Q3,2018:Q4,Total GDP
0,United States *,All industry total,20163159.0,20510177.0,20749752.0,20897804.0,164641784.0
1,United States *,Private industries,17654891.0,17977475.0,18189960.0,18320015.0,144284682.0
2,United States *,"Agriculture, forestry, fishing and hunting",169446.0,172100.0,160510.0,163799.0,1331710.0
3,United States *,"Mining, quarrying, and oil and gas extra...",328770.0,346432.0,361288.0,350091.0,2773162.0
4,United States *,Utilities,322592.0,326273.0,323103.0,331440.0,2606816.0


In [26]:
all_industries = new_df[new_df['Description'] == 'All industry total']

In [27]:
all_industries.reset_index()

,index,GeoName,Description,2018:Q1,2018:Q2,2018:Q3,2018:Q4,Total GDP
0,0,United States *,All industry total,20163159.0,20510177.0,20749752.0,20897804.0,164641784.0
1,27,Alabama,All industry total,217369.7,220572.2,223140.7,225859.4,1773884.0
2,51,Alaska,All industry total,53563.0,54372.7,55410.0,55590.6,437872.6
3,75,Arizona,All industry total,340672.6,345474.0,351708.8,355332.9,2786376.6
4,99,Arkansas,All industry total,126088.8,128352.5,129032.8,130201.5,1027351.2
5,123,California,All industry total,2928096.5,2992622.8,3017340.5,3052871.3,23981862.2
6,147,Colorado,All industry total,362721.2,370476.6,375311.6,378488.8,2973996.4
7,171,Connecticut,All industry total,272854.5,273884.5,278294.5,277874.1,2205815.2
8,195,Delaware,All industry total,72091.2,73024.7,74461.4,74348.1,587850.8
9,219,District of Columbia,All industry total,138469.2,139579.6,141968.8,142626.2,1125287.6


In [28]:
ind_df = all_industries.drop(all_industries.index[[0, 52, 53, 54, 55, 56, 57, 58, 59]]).reset_index()

clean_ind = ind_df.reset_index()

ind_df_clean = clean_ind.drop(columns = ['index', 'level_0'])

ind_df_clean


,GeoName,Description,2018:Q1,2018:Q2,2018:Q3,2018:Q4,Total GDP
0,Alabama,All industry total,217369.7,220572.2,223140.7,225859.4,1773884.0
1,Alaska,All industry total,53563.0,54372.7,55410.0,55590.6,437872.6
2,Arizona,All industry total,340672.6,345474.0,351708.8,355332.9,2786376.6
3,Arkansas,All industry total,126088.8,128352.5,129032.8,130201.5,1027351.2
4,California,All industry total,2928096.5,2992622.8,3017340.5,3052871.3,23981862.2
5,Colorado,All industry total,362721.2,370476.6,375311.6,378488.8,2973996.4
6,Connecticut,All industry total,272854.5,273884.5,278294.5,277874.1,2205815.2
7,Delaware,All industry total,72091.2,73024.7,74461.4,74348.1,587850.8
8,District of Columbia,All industry total,138469.2,139579.6,141968.8,142626.2,1125287.6
9,Florida,All industry total,1020176.9,1032277.4,1048477.6,1056013.7,8313891.2
